<a href="https://colab.research.google.com/github/ds4geo/ds4geo/blob/master/DS4GEO_L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Science for Geoscientists - Winter Semester 2020**
# **Session 2**

In the previous session, we handled data in a very simple way using pandas. In this session we will introduce a few other helpful python object types for handling data, and expecially learn how to index/slice data (extract only certain parts of the data/object). Specifically, we will cover lists, dictionaries, and arrays from the numpy library.

We will also introduce simple array operations and aggregations, then apply these topics to a worked example from the geosciences.





# Section 1 - Lists, Dictionaries and Indexing

Lists and dictionaries are built-in python objects useful for storing and handling data.

# Lists
Python lists are ordered collections of other python objects, separated by commas. They are defined by square brackets [ ]

In [ ]:
a = [1,2,3] # List of integers
print("a:", a)

b = [1.5, 2.5, 3.5] # List of floats
print("b:", b)

In [ ]:
# Lists can contain different types
c = [1, "data", 2.5]
print("b:", b)

# Including other lists (nested)
d = [[1,2,3], [4,5,6]]
print("d:", d)

e = [a, b]
print("e:", e)

In [ ]:
# They can contain any other python objects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
f = [pd, np, plt] # But there's no reason to actually do this
print("f:", f)

In [ ]:
# From last week, you'll recall dir() can be used to find methods on objects
a = [1, 2, 3]
# Append adds a new item to the end of a list
a.append(4)
print("a:", a)

# Extend joins to lists *in place*
a.extend(b) # notice we don't assign a result
print("a:", a)
# + operator when applied to two lists but not *in place*:
h = a + b
print("h:", h)

# sort does what it suggests, in place
a.sort()
print("a:", a)

In [ ]:
# Tuples are another type very similar to lists except they can't be modified
# i.e. you cannot append to a tuple
# They are defined by parentheses ( ) instead of [ ]
a_tuple = (1, 2, 3)
print("a_tuple:", a_tuple)

# The specific reasons for using tuples complex.
# You will see them in documentation, but usually you can just use a list

# Dictionaries
Python dictionaries are un-ordered collections of pairs known as keys and values. They function like language dictionaries where you look up a word (they key) and see its definition or translation (value).
They are defined with braces { }, separated by commas, and colons : indicate the key-value relationships.

In [ ]:
# Create a simple German to English language dictionary
De2En = {"Bier": "Beer", "Wurst": "Sausage"}

# When making lists and dictionaries, you can wrap between lines for readability:
De2En = {"Bier": "Beer",
         "Wurst": "Sausage",
         "Kren": "Horseradish"}


In [ ]:
# Values can be any python object, e.g. lists:
rocks = {"igneous": ["Granite", "Basalt", "Rhyolite"],
         "Sedimentary": ["Sandstone", "Limestone"]}

# Keys can be some python objects (int, float, string, tuple), but not others (lists or dicts)
# Keys and values do not all have to be the same type
complex_dict = {0: "zero",
                "one point 5": 1.5,
                2.5: ["two", "point", "five"]}

# Dictionaries can also be nested like lists.
# Note the nesting is multi-line and aligned to improve readability
rock_dict = {"granite": {"type": "igneous",
                         "composition": {"quartz": 0.5,
                                         "feldspar": 0.2},
                         "locations": [(50.59671,-3.98289),
                                       (50.59591,-4.61987)]},
             "sandstone": {}}


# List and Dictionary Indexing
You can select objects/data from lists and dictionaries using square brackets [ ].
List indexing is based on numeric positions, while dictionary indexing is based on its keys.

**Note** python positional indexing (for lists, numpy, pandas, etc) always starts at 0. i.e. the first item is 0. This might seem counter intuitive at first, but when combined with some other features of python, it actually simplifies code in many situations!

In [ ]:
# Remind ourselves what is in variable "c"
print(c)

# Print positions 0, 1 and 2 of list "c"
print(c[0])
print(c[1])
print(c[2])

In [ ]:
# If we try to index a position beyond the size of the list, we get an index error
# Uncomment this line to try it.
# (It is commented out so you can run all the code in this notebook without generating an error)
#print(c[3])

In [ ]:
# List indexing also works with negative numbers in reverse, with -1 being the last index
print(c)
print(c[-1]) # the last item in c

In [ ]:
# With nested objects, indexing can be stacked with sets of square brackets [ ][ ]
print(d)
print(d[1])
print(d[1][2])

In [ ]:
# Indexing tuples and strings works exactly the same way
print(a_tuple)
print(a_tuple[0])

print(c)
print(c[1])
print(c[1][2])

In [ ]:
# For lists, tuples and strings (and numpy - see later), ranges also work.
# Ranges are "half-open", i.e. include the first index, but not the last.
# This is so when you use a range of e.g. 2:4, you get a result of length 2, despite indexing starting at 0
print(a)
print(a[2:4])

In [ ]:
# Also useful is finding the length of lists, dicts and strings:
print("length of list a:", len(a))
print("length of dict rocks:", len(rocks))
print("length of string in position 1 of list c:", len(c[1]))

In [ ]:
# Dictionaries are indexed by their keys:
print(De2En["Bier"])

# And example of indexing nested objects
print(rocks["igneous"])
print(rocks["igneous"][1])
print(rock_dict["granite"])
print(rock_dict["granite"]["composition"])
print(rock_dict["granite"]["composition"]["quartz"])

In [ ]:
# You can also expand dictionaries using indexing assignment:
De2En["Semmel"] = "Bread roll"
print(De2En)

rocks["metamorphic"] = ["Gneiss, Schist"]
print(rocks)

# And you can use methods on the objects indexed:
rocks["igneous"].append("Gabbro")
print(rocks)

# Section 2.2 - Numpy part 1
Last week we used the popular python library Pandas, but didn't introduce it formally.
This week we will also be using a popular libary called Numpy.
Pandas is built upon Numpy, and they work well together.
Pandas is good at data handling, manipulation and analysis, while Numpy is the basis of numerical operations and processing.
See more here:
* https://pandas.pydata.org/
* https://numpy.org/

We will use both Pandas and Numpy throughout the course. Together (along with matplotlib), they are the basis of Data Science in python.

Numpy is based around multi-dimensional arrays (of data), and allows efficient indexing, operations and aggregation of said arrays.
For those not familiar with multi-dimensional arrays (also called nd-arrays), imagine an excel spreadsheet as a 2 dimensional table/array with rows and columns, but that you can have as many dimensions as you like.

As an example, in satellite remote sensing, it is typical to have a time-series of many multi-band (e.g. red, green, blue, infra-red) images. Therefore, you might have an array of 4 dimensions: [pixel rows, pixel columns, time, band]. So for each x-y pixel, at each point in time, you have a value for each band.

In the following section, we will create arrays, learn how to do simple operations on them and perform basic aggregations. In the following section, we will explore Numpy's powerful indexing system.

The website Datacamp.com provides an excellent Numpy "cheat-sheet". It is highly recommended to keep it handy when working with Numpy, and going through it in your own time.
https://www.datacamp.com/community/blog/python-numpy-cheat-sheet



## 2.2.1 - Creating Arrays

In [ ]:
# Here we cover simple ways to create numpy arrays.
# We will cover loading and importing data, e.g. from pandas later.

# The simplest way to create an array is from a list
array = np.array([1,2,3])
print(array)

# Or with nested lists for multiple dimensions
array_2d = np.array([[1,2,3],[4,5,6]])
print(array_2d)

In [ ]:
# numpy provides some functions to create arrays by shape:
# make a 1d array of 5 zeros
array_zeros = np.zeros(5) 
print(array_zeros)

# Make a 2d array of 1s
array_ones = np.ones((2,5))
print(array_ones)

# numpy arrays have an attribute shape:
print("array_zeros size:", array_zeros.shape)
print("array_ones size:", array_ones.shape)

In [ ]:
# Create an array of consecutive integers in a range using np.arange
arange_1 = np.arange(15,25)
print(arange_1)

# Use arange to create larger steps
arange_2 = np.arange(15,25,2)
print(arange_2)

# If one needs a standard python list in this style:
print(range(5))

In [ ]:
# Create array across range by number of intermediate steps, rather than the step itself
linspace_1 = np.linspace(0,4,17)
print(linspace_1)

In [ ]:
# Arrays of random numbers can be produced with np.random.random_sample np.random.standard_normal
uni_random = np.random.random_sample(10)
print(uni_random)

np.random.standard_normal()
norm_random = np.random.standard_normal(10)
print(norm_random)

## 2.2.2 - Operations

In [ ]:
# Python lets us do operations on integers and floats
print(1+2)
print(2*3)
print(2.5*5)
print(2**6)
print(64/4)

In [ ]:
# But on lists, these operators do other things:
print([1,2,3] + [4]) # List concatenation
print([1,2,3] * 3) # List duplication
# Operators like / and - do not work

In [ ]:
# Operators can be applied to numpy arrays in an intuitive way:
# Operators between a numpy array and a single int or float apply the operation to all elements in the array:
a = np.ones(5)
b = np.arange(5)

print("a:",a)
print("a + 1:",a + 1)
print("a - 1:",a - 1)
print("a * 2:",a * 2)
print("a / 2:",a / 2)

print("b * 2:", b * 2)

In [ ]:
# Operations between arrays of the same shape result are element-wise:
print("b:",b)
print("b * b:", b * b)

## 2.2.3 - Aggregations

# Section 3 - Numpy Excercise 1

In [ ]:
# Create the following sequences as numpy arrays:
# 1. [3, 6, 9, 12, ...., 99]
# 2. [15, 15, 15, 15, 15, 15]
# 3. [0, 0.5, 1, 1.5, ...., 100]


In [ ]:
# Create the following arrays:
# 1. 1d array of size 100 with random decimal numbers between 1 and 100
# 2. 1d array of size 50 with random integers between 25 and 75
# 3. 1d array of size 100 with normal (gaussian) distributed numbers with a mean of 5 and a standard deviation of 2


In [ ]:
# Challenges
# 1. [0,1,0,2,0,4,0,8,0,16,0,32,0,64,.....65536]
#   Note, this can be done in many different ways, including with other numpy functions
#   and using numpy indexing, but it is possible to do with only the functions described above.
# 2. An array representing the sum of rolling a pair of 6 sided dice 1000 times (if your game of monopoly overruns more than usual)

In [ ]:
(2 ** np.arange(-0.5,50.5,0.5)) * np.array([0,1]*51)

# Section 3 - Numpy 2

Indexing, simple broadcasting and booleans (first)

# Section 4 - LA-ICPMS data reduction excercise
*Write this, then check all requirements are fulfilled above*

In the geosciences, we often have raw measurement data from an analytical machine, and need to convert or "reduce" that data to make it useful for further analysis and interpretation. A very common example is conversion of mass spectrometer (or similar) raw count data to composition data, such as weight percentage or ppm of the analysis material. In many cases there are specific software packages to perform this data reduction without needing to do any coding, but frequently the underlying methodology is not complex and could be easily done in python. As an example, in this excercise we will convert raw Laser Ablation - ICP Mass Spectrometer (LA-ICPMS) data to mass fraction of the sample material.

The following paper explains LA-ICPMS, typical data reduction proceedures and software packages:
https://www.sciencedirect.com/science/article/abs/pii/S0009254118305461?via%3Dihub

Using python, we will perform steps 2 to 5 of the "Basic Processing" in section 2.1 of that paper.

**Note: Each field (within and beyond the geosciences) has its own literature about data reduction and processing. You should consult authoritative sources when doing this work to avoid methodological errors. This excercise is intended to demonstrate that the mathematical and programming required is easily achievable with only basic python knowledge.***

**Data Reduction Steps**:
* 1. Load the data
* 2. Identification of background, samples and standards in the raw data
* 3. Apply background correction
* 4. Standardise data
* 5. Calibrate data to standards
* 6. Calculate the mass fraction


## 2.4.1 - Load data

The example data we will use is from the testing datasets for a python tool for LA-ICPMS data reduction:
(https://github.com/oscarbranson/latools).

In [ ]:
# Load the data from here as a pandas data frame:
# https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/laicpms_sample.csv


In [ ]:
# Create:
# 1. a 1d numpy array of the "Time" column
# 2. a 2d numpy array of the element count columns

In [ ]:
# ANSWERS
data_pd = pd.read_csv("https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/laicpms_sample.csv", header=1)
data = data_pd.to_numpy()
time = data[:,0]
raw_te = data[:,1:]

In [ ]:
# Make a/some plots of the data to get an overview

## 2.4.2 - Identify background, samples and standards

When you plot the data, you will see several periods where the counts (for any element) are well above 0. The first 3 of these sections are standards, the last 4 are samples. The intermediate parts are background.

We need to identify the time intervals corresponding to samples, sections and background for the following analyses. We will do this by identifying the start and end times/positions (here, position is measured in time) for each relevant section. We then create a boolean index array for each.

It is recommended to work together with your classmates to complete this task by sharing the start and end positions.

In [ ]:
# Find out and record the start and end positions of at least 2 background sections
# Record them in a list containing dictionaries with the following format:
# [{"start": <position in seconds>, "end": <position in seconds>}
#  {"start": <>, "end": <>},
#   .....]

In [ ]:
# Do the same for all the sample sections, and separately for all the standards sections

In [ ]:
# Create a boolean index array for each of background, samples and standards.
# Each should be same shape as the time array (use the .shape method to check).
# hint: use np.logical_and to create a boolean index for each section,
#       then combine the results with np.any. 

In [ ]:
# ANSWERS
bg_loc = [{"start": 0, "end": 25},
          {"start": 491, "end": 498}]
stand_loc = [{"start": 27, "end": 82},
            {"start": 105, "end": 160},
            {"start": 184, "end": 220},
            ]
samp_loc = [{"start": 269, "end": 340 },
            {"start": 363, "end": 389},
            {"start": 409, "end": 433},
            {"start": 453, "end": 487}]

bg_idx = np.any([np.logical_and(time > bg_loc[0]["start"], time < bg_loc[0]["end"]),
                 np.logical_and(time > bg_loc[1]["start"], time < bg_loc[1]["end"])],
                axis=0)

stand_idx = np.any([np.logical_and(time > stand_loc[0]["start"], time < stand_loc[0]["end"]),
                 np.logical_and(time > stand_loc[1]["start"], time < stand_loc[1]["end"]),
                 np.logical_and(time > stand_loc[2]["start"], time < stand_loc[2]["end"])],
                axis=0)

samp_idx = np.any([np.logical_and(time > samp_loc[0]["start"], time < samp_loc[0]["end"]),
                 np.logical_and(time > samp_loc[1]["start"], time < samp_loc[1]["end"]),
                 np.logical_and(time > samp_loc[2]["start"], time < samp_loc[2]["end"])],
                axis=0)


## 2.4.3 - Apply background correction
The background counts should be removed from the rest of the signal for each element.

We therefore take the average counts during the background periods for each element and subtract these values from the element arrays.

In [ ]:
# Create an array of the average (mean) counts for each element during the background sections.


In [ ]:
# Subtract the per-element backgrounds from the entire dataset

In [ ]:
# ANSWER
bg_vals = np.mean(raw_te[bg_idx], axis=0)
bg_corr = raw_te - bg_vals

##2.4.4 - Standardize data
In LA-ICPMS analysis, the amount of analyte which is measured depends on how much is ablated by the laser, which in turn depends on the material properties of the sample (known as matrix effects). To remove matrix effects and other spurious features, we can standardize the data to an element which we expect to be present at a constant concentration. For carbonates, an isotope of Ca is often used and is known as the internal standard.

Standardization in this case means that we convert all other element data to count ratios of that element vs Ca44.

In [ ]:
# Convert all data into ratios to Ca44 - i.e. divide all element counts by Ca44 counts

In [ ]:
#ANSWER
castd_te = bg_corr / bg_corr[:,4:5]

##2.4.5 - Calibrate data
Next we calibrate the count ratios to composition ratios. We do this using the measured standards of known composition.

Preparation of the standard composition in terms of count ratio is beyond the scope of this excercise, so the required data is provided ready-to-use. In this case, we will only use one of the standards, but more complex methods exist to simultaneously calibrate using multiple standards and to thereby improve estimation of the measurement uncertainty.


In [ ]:
# Load the standard composition from here:
# https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/laicpms_standard.csv
# Convert the dataframe to a numpy array and check its shape

In [ ]:
# The calibration data refers only to the first measured standard.
# Create an boolean index array for this standard.
# Then calculate the mean values per element for the standard

In [ ]:
# Calculate the conversion ratio by dividing calibration data by the measured (standardized) standard data
# Apply the conversion ratio by dividing the standardized element data by the calibration



In [ ]:
# Remove all data except the samples
# Assign everything else to np.nan

In [ ]:
# Make some plots to visualise the output.
# Optionally you can convert the calibrated data back to a pandas DataFrame

In [ ]:
# ANSWER
std_dat = pd.read_csv(r"https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/laicpms_standard.csv",sep=",").to_numpy()
#cali_te = castd_te / std_dat
stand_1_idx = np.logical_and(time > stand_loc[0]["start"], time < stand_loc[0]["end"])
stand_comp = np.mean(castd_te[stand_1_idx,:], axis=0)
calibr = std_dat / stand_comp
calibrated = castd_te / calibr
calibrated[~samp_idx] = np.nan

In [ ]:
for c,l in enumerate(data_pd.columns[1:]):
  plt.plot(time,calibrated[:,c], label=l)
plt.legend()

##2.4.6 Calibrate mass fraction
We standardized our data to an internal standard of constant concentration (i.e. a ratio of Ca44). However, if we want to calculate the mass fraction, we need to convert our data back from that ratio to the mass fraction. To do this, we need to know the concentration of the internal standard in the material. This needs to be separately measured or assumed.

Here we will make an assumption.

In [ ]:
# Estimate mass fraction of Ca44 in sample (a foram)
# Assume the foram is entirely composed of CaCO3
# 1. Calculate the weight fraction of Ca in CaCO3
